<a href="https://colab.research.google.com/github/alexacheui/ateema_capstone/blob/main/Control_Center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo --quiet
!pip install flask pyngrok --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.8 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
import os
from google.colab import userdata
from pyngrok import ngrok
import requests
import json
import time
import threading

class SurveyApp:
    def __init__(self, mongo_uri):
        try:
            self.client = MongoClient(mongo_uri)
            self.db = self.client.ateema
            self.collection = self.db['survey']
            print("Successfully connected to MongoDB.")
        except Exception as e:
            print(f"Error connecting to MongoDB: {e}")
            raise

    def process_survey(self, request_data):
        print("**Received Data:**")
        print(request_data)

        def join_labels(labels):
            return ', '.join([label.split()[0] for label in labels])

        try:
            answers = request_data['form_response']['answers']
            survey_data = {
                'first_name': answers[0]['text'],
                'last_name': answers[1]['text'],
                'phone_number': answers[2]['phone_number'],
                'email': answers[3]['email'],
                'travel_city': answers[4]['choice']['label'],
                'start_date': answers[5]['date'],
                'end_date': answers[6]['date'],
                'group_type': answers[7]['choice']['label'],
                'group_size': answers[8]['number'],
                'group_age': answers[9]['choice']['label'],
                'pref_dining': join_labels(answers[10]['choices']['labels']),
                'pref_beverage': join_labels(answers[11]['choices']['labels']),
                'pref_entertainment': join_labels(answers[12]['choices']['labels']),
                'pref_cultural': join_labels(answers[13]['choices']['labels']),
                'pref_outdoor': join_labels(answers[14]['choices']['labels']),
                'pref_education': join_labels(answers[15]['choices']['labels']),
                'pref_shop': join_labels(answers[16]['choices']['labels']),
                'comment': answers[17]['text'],
                'video_length': answers[18]['choice']['label'],
            }

            df = pd.DataFrame([survey_data])
            print("**Created DataFrame:**")
            print(df)

            records = df.to_dict(orient='records')
            self.collection.insert_many(records)

            # Start the recommendation and video generation process in a separate thread
            threading.Thread(target=self.process_recommendation_and_generate_video, args=(df,)).start()

            return True
        except Exception as e:
            print(f"Error processing survey data: {e}")
            return False

    def get_recommendation(self, df):
        recommendation_url = 'https://2dff-34-73-209-198.ngrok-free.app/webhook'
        response = requests.post(recommendation_url, json=df.to_dict(orient='records')[0])
        if response.status_code == 200:
            print("Recommendation request successful!")
            print("Response content:")
            print(json.dumps(response.json(), indent=2))
            return response.json().get('recommendation', {})
        else:
            print(f"**Failed to get recommendation. Status code: {response.status_code}**")
            return None

    def process_recommendation_and_generate_video(self, df):
        recommendation = self.get_recommendation(df)
        if recommendation:
            self.generate_video(recommendation)

    def generate_video(self, recommendation):
        video_url = 'https://0613-34-28-72-113.ngrok-free.app/webhook'
        video_response = requests.post(video_url, json=recommendation)

        if video_response.status_code == 202:
            job_id = video_response.json().get('job_id')
            print(f"Video processing started. Job ID: {job_id}")

            max_attempts = 40  # 40 * 120 seconds = 80 minutes
            time.sleep(10)  # Add a slight delay before the first status check
            for attempt in range(max_attempts):
                print(f"Attempt {attempt + 1}/{max_attempts}: Checking status for job ID {job_id}")
                status_response = requests.get(f"{video_url}/status/{job_id}")
                if status_response.status_code == 200:
                    job_status = status_response.json()
                    if job_status['status'] == 'completed':
                        video_link = job_status['result']
                        print("Video generation completed!")
                        print(f"Video URL: {video_link}")
                        return video_link
                    elif job_status['status'] == 'failed':
                        print("Video generation failed.")
                        print(f"Error: {job_status['result']}")
                        return None
                else:
                    print(f"Failed to check status. Status code: {status_response.status_code}")
                    print(f"Response content: {status_response.content}")

                if attempt == max_attempts - 1:
                    print("Video generation timed out.")
                else:
                    print("Video still processing. Checking again in 120 seconds...")
                    time.sleep(120)  # Wait for 120 seconds before checking again
        else:
            print(f"Failed to start video generation. Status code: {video_response.status_code}")
            return None

app = Flask(__name__)
mongo_uri = userdata.get('mongodb_key')
survey_app = SurveyApp(mongo_uri)

@app.route("/")
def index():
    return "<h1>Hello World</h1>"

@app.route('/webhook', methods=['POST'])
def typeform_webhook():
    request_data = request.get_json()
    success = survey_app.process_survey(request_data)
    if success:
        return jsonify({'message': 'Survey data received and processing started'}), 200
    else:
        return jsonify({'message': 'Error processing survey data'}), 500

if __name__ == '__main__':
    ngrok.set_auth_token(userdata.get('ngrok_key_3'))
    ngrok_tunnel = ngrok.connect(5000)
    print('Control Center URL:', ngrok_tunnel.public_url)
    app.run(port=5000)


Successfully connected to MongoDB.
Control Center URL: https://3b59-34-90-198-135.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


**Received Data:**
{'event_id': '01J51SXB06C5S4T93VPW5ZQ2EJ', 'event_type': 'form_response', 'form_response': {'form_id': 'FkcBezxr', 'token': '01J51SXB06C5S4T93VPW5ZQ2EJ', 'landed_at': '2024-08-11T22:45:58Z', 'submitted_at': '2024-08-11T22:45:58Z', 'definition': {'id': 'FkcBezxr', 'title': 'My new form', 'fields': [{'id': 'cyC2jup2h9qd', 'ref': '596e32db-f5ef-422a-85b0-1e64723cff81', 'type': 'short_text', 'title': 'First name', 'properties': {}}, {'id': 'Ap9DQevjbydK', 'ref': '8b4f0707-c312-4658-94b0-d215a63fced2', 'type': 'short_text', 'title': 'Last name', 'properties': {}}, {'id': 'eZm0OtqIP460', 'ref': '600b2743-94a4-49bd-a9e9-2651313a9c5f', 'type': 'phone_number', 'title': 'Phone number', 'properties': {}}, {'id': 'RWH1jVaQrbdz', 'ref': 'dfe478d9-e2e2-4e75-9623-6280b9567724', 'type': 'email', 'title': 'Email', 'properties': {}}, {'id': 'rAKa2WQMixJL', 'ref': '86a2d1bb-5371-47bd-9e59-8556125ea162', 'type': 'picture_choice', 'title': '*Travel is always exciting! which town are you 

INFO:werkzeug:127.0.0.1 - - [11/Aug/2024 22:45:58] "POST /webhook HTTP/1.1" 200 -


Recommendation request successful!
Response content:
{
  "message": "Confirm received data",
  "recommendation": {
    "recommendation": "{ \"Hello Lorem! I'm Ateema, your AI tour guide. You're planning a trip to Chicago from 2017-02-06 to 2017-02-06 with a group of 42 people aged 20 and above. Your preferences include gourmet dining, outdoor activities, family-friendly entertainment, and more. Based on your interests, I've curated some recommendations for you. Here are my suggestions: **Dining:** 1. **Alinea**: A Michelin three-star restaurant offering an avant-garde culinary experience; 2. **Girl & the Goat**: A Mediterranean-inspired restaurant from Top Chef winner Stephanie Izard, perfect for a group outing. **Beverages:** 1. **The Violet Hour**: A stylish cocktail bar with creative drinks and a speakeasy vibe; 2. **Big Jones**: A rooftop bar offering craft cocktails and stunning city views. **Entertainment:** 1. **Second City**: A renowned comedy club featuring improv performances